In [1]:
import pandas as pd
pd.set_option("max_columns", None)
wargame = pd.read_csv("../data/510049986/data.csv", index_col=0)

In [2]:
%matplotlib inline
import seaborn as sns

First of all, a quick sampling shows that `AliasName` is not a reliable indicator for names, as many names are given as nulls. This is due to localization: units whose names are localized cross-region have a special `LocalizationManager` to deal with that, which I didn't import into this dataset.

In [9]:
wargame['AliasName'].sample(10)

1686                  null
1791                  null
766                   IGLA
1599                  null
818            9K33M2 ROMB
1626                  null
142     BRDM-2  STRZALA-1M
1046         T-55AM MERIDA
1082                 T-80B
546           LEOPARD AS1+
Name: AliasName, dtype: object

In [15]:
wargame[wargame['AliasName'].map(lambda n: 'GR3' in n)]

,AcknowUnitType,AliasName,AutoOrientation,CanDeploySmoke,CanWinExperience,Category,ClassNameForDebug,CoutEtoile,CyclicManoeuvrability,DamageModuleID,DamageSelectorID,DeathExplosionAmmo,DescriptorId,EliteDeployableAmount,ExperienceGainBySecond,ExperienceModuleID,ExperienceSelectorID,Factory,FlyingAltitude,FuelCapacity,FuelModuleSelectorID,FuelMoveDuration,GFactorLimit,GunMuzzleSpeed,HardenedDeployableAmount,HitRollECMModifier,HitRollSizeModifier,IconeType,IsPrototype,IsTargetableAsBoat,Key,KillExperienceBonus,LateralSpeed,ManageUnitOrientation,Mass,MaxAcceleration,MaxDamages,MaxDeceleration,MaxHPForHUD,MaxInclination,MaxPacks,Maxspeed,MinimalAltitude,MotherCountry,MovementModuleID,MovementSelectorID,MovementType,NameInMenuToken,Nationalite,PositionInMenu,ProductionPrice,ProductionTime,ProductionYear,RookieDeployableAmount,RotorArea,SalvoWeaponOneIsMain,SalvoWeaponThreeIsMain,SalvoWeaponTwoIsMain,SalvosWeaponOne,SalvosWeaponThree,SalvosWeaponTwo,SpeedBonusOnRoad,StickToGround,TempsDemiTour,TerrainsToIgnoreMask,TextureForInterface,TextureMotherCountryForInterface,TextureTransportForInterface,TorqueManoeuvrability,TrainedDeployableAmount,Transporter,TurretNoneID,TurretNoneType,TurretOneID,TurretOneType,TurretThreeID,TurretThreeType,TurretTwoID,TurretTwoType,TypeForAcknow,UnitDescriptorID,UnitMovingType,UnitStealthBonus,UpgradeRequire,UpwardsSpeed,VehicleSubType,VeteranDeployableAmount,VisibilityModuleID,VisibilitySelectorID,VitesseCombat,WeaponFourAffecteParNombre,WeaponFourAffichageMenu,WeaponFourAffichageMunitionParSalve,WeaponFourAmmunitionID,WeaponFourAngleDispersion,WeaponFourArme,WeaponFourCaliber,WeaponFourCorrectedShotDispersionMultiplier,WeaponFourDescriptorId,WeaponFourDispersionAtMaxRange,WeaponFourDispersionAtMinRange,WeaponFourEfficaciteSelonPortee,WeaponFourFX_frottement,WeaponFourFX_tir_sans_physic,WeaponFourFX_tir_tendu,WeaponFourFX_vitesse_de_depart,WeaponFourFireDescriptor,WeaponFourFireTriggeringProbability,WeaponFourGuidance,WeaponFourHitRollRule,WeaponFourIgnoreInflammabilityConditions,WeaponFourInterdireTirReflexe,WeaponFourInterfaceWeaponTexture,WeaponFourIsFireAndForget,WeaponFourIsSubAmmunition,WeaponFourLevel,WeaponFourMissileDescriptor,WeaponFourMissileTimeBetweenCorrections,WeaponFourName,WeaponFourNbTirParSalves,WeaponFourNbrProjectilesSimultanes,WeaponFourNeedModelChange,WeaponFourNoiseDissimulationMalus,WeaponFourPhysicalDamages,WeaponFourPorteeMaximale,WeaponFourPorteeMaximaleBateaux,WeaponFourPorteeMaximaleHA,WeaponFourPorteeMaximaleProjectile,WeaponFourPorteeMaximaleTBA,WeaponFourPorteeMinimale,WeaponFourPorteeMinimaleBateaux,WeaponFourPorteeMinimaleHA,WeaponFourPorteeMinimaleProjectile,WeaponFourPorteeMinimaleTBA,WeaponFourProjectileType,WeaponFourPuissance,WeaponFourRadiusSplashPhysicalDamages,WeaponFourRadiusSplashSuppressDamages,WeaponFourRandomDispersion,WeaponFourRayonPinned,WeaponFourSmokeDescriptor,WeaponFourSupplyCost,WeaponFourSuppressDamages,WeaponFourTempsAnimation,WeaponFourTempsDeVisee,WeaponFourTempsEntreDeuxFx,WeaponFourTempsEntreDeuxSalves,WeaponFourTempsEntreDeuxTirs,WeaponFourTirIndirect,WeaponFourTirReflexe,WeaponFourTypeArme,WeaponFourTypeName,WeaponFourWeaponCursorType,WeaponManagerID,WeaponManagerModuleSelectorID,WeaponOneAffecteParNombre,WeaponOneAffichageMenu,WeaponOneAffichageMunitionParSalve,WeaponOneAmmunitionID,WeaponOneAngleDispersion,WeaponOneAnimateOnlyOneSoldier,WeaponOneArme,WeaponOneCaliber,WeaponOneCorrectedShotDispersionMultiplier,WeaponOneDescriptorId,WeaponOneDispersionAtMaxRange,WeaponOneDispersionAtMinRange,WeaponOneEfficaciteSelonPortee,WeaponOneFX_frottement,WeaponOneFX_tir_sans_physic,WeaponOneFX_tir_tendu,WeaponOneFX_vitesse_de_depart,WeaponOneFireDescriptor,WeaponOneFireTriggeringProbability,WeaponOneGuidance,WeaponOneHitRollRule,WeaponOneIgnoreInflammabilityConditions,WeaponOneInterdireTirReflexe,WeaponOneInterfaceWeaponTexture,WeaponOneIsFireAndForget,WeaponOneIsSubAmmunition,WeaponOneLevel,WeaponOneMissileDescriptor,WeaponOneMissileTimeBetweenCorrec

Which is all fine and good, but some of the `ClassNameForDebug` entries are not very good either. What does `ROK_BlackBerets` mean!

485 units are localized and 1333 are not. The most fruitful approach would be to merge these two, semantically, prefering a lowercased version of AliasName when one is available.

In [20]:
wargame['AliasName'].value_counts()[0], wargame['AliasName'].value_counts()[1:].sum()

(485, 1333)

In [26]:
def name(srs):
    if srs['AliasName'] != 'null':
        return srs['AliasName'].title()
    else:
        return srs['ClassNameForDebug'].replace("Unit", "").replace("_", " ")

names = wargame.apply(name, axis='columns')

In [27]:
names

0                            Dana
1                       Rzp Kub-M
2                 2K22 Tunguska-M
3                   2K22 Tunguska
4                      2S15 Norov
5                     2S19 Msta-S
6                     2S1 Gozdzik
7                    2S1 Gvozdika
8                   2S23 Nona-Svk
9                   2S3M Akatsiya
10                   2S3 Akatsiya
11                     2S7M Malka
12                       2S7 Pion
13                    2S7 Piwonia
14                       2S9 Nona
15                    Groupe Cmdt
16                         Deprec
17                    Highlanders
18                 Motostrelki 90
19                    9K33M3 Romb
20                       9K33 Osa
21                    9K33 Osa-Ak
22                   9K33 Osa-Akm
23               Mt-Lb Strela-10M
24                       9K37 Buk
25                    9K37 Buk-M1
26               Aml 60/20 Serval
27                           Amos
28                  Amx-105 Mk.61
29            

In [28]:
wargame['UnitName'] = names

A more general op, get rid of all of those "null" strings!

In [31]:
import numpy as np
wargame = wargame.replace(to_replace='null', value=np.nan)

We need to massively reorder things. This is not trivial, and takes some thought, as there's over 300 variables!

In [35]:
print(list(wargame.columns))

['AcknowUnitType', 'AliasName', 'AutoOrientation', 'CanDeploySmoke', 'CanWinExperience', 'Category', 'ClassNameForDebug', 'CoutEtoile', 'CyclicManoeuvrability', 'DamageModuleID', 'DamageSelectorID', 'DeathExplosionAmmo', 'DescriptorId', 'EliteDeployableAmount', 'ExperienceGainBySecond', 'ExperienceModuleID', 'ExperienceSelectorID', 'Factory', 'FlyingAltitude', 'FuelCapacity', 'FuelModuleSelectorID', 'FuelMoveDuration', 'GFactorLimit', 'GunMuzzleSpeed', 'HardenedDeployableAmount', 'HitRollECMModifier', 'HitRollSizeModifier', 'IconeType', 'IsPrototype', 'IsTargetableAsBoat', 'Key', 'KillExperienceBonus', 'LateralSpeed', 'ManageUnitOrientation', 'Mass', 'MaxAcceleration', 'MaxDamages', 'MaxDeceleration', 'MaxHPForHUD', 'MaxInclination', 'MaxPacks', 'Maxspeed', 'MinimalAltitude', 'MotherCountry', 'MovementModuleID', 'MovementSelectorID', 'MovementType', 'NameInMenuToken', 'Nationalite', 'PositionInMenu', 'ProductionPrice', 'ProductionTime', 'ProductionYear', 'RookieDeployableAmount', 'Roto

In [17]:
wargame[wargame['ClassNameForDebug'].map(lambda n: 'Beret' in n)]

,AcknowUnitType,AliasName,AutoOrientation,CanDeploySmoke,CanWinExperience,Category,ClassNameForDebug,CoutEtoile,CyclicManoeuvrability,DamageModuleID,DamageSelectorID,DeathExplosionAmmo,DescriptorId,EliteDeployableAmount,ExperienceGainBySecond,ExperienceModuleID,ExperienceSelectorID,Factory,FlyingAltitude,FuelCapacity,FuelModuleSelectorID,FuelMoveDuration,GFactorLimit,GunMuzzleSpeed,HardenedDeployableAmount,HitRollECMModifier,HitRollSizeModifier,IconeType,IsPrototype,IsTargetableAsBoat,Key,KillExperienceBonus,LateralSpeed,ManageUnitOrientation,Mass,MaxAcceleration,MaxDamages,MaxDeceleration,MaxHPForHUD,MaxInclination,MaxPacks,Maxspeed,MinimalAltitude,MotherCountry,MovementModuleID,MovementSelectorID,MovementType,NameInMenuToken,Nationalite,PositionInMenu,ProductionPrice,ProductionTime,ProductionYear,RookieDeployableAmount,RotorArea,SalvoWeaponOneIsMain,SalvoWeaponThreeIsMain,SalvoWeaponTwoIsMain,SalvosWeaponOne,SalvosWeaponThree,SalvosWeaponTwo,SpeedBonusOnRoad,StickToGround,TempsDemiTour,TerrainsToIgnoreMask,TextureForInterface,TextureMotherCountryForInterface,TextureTransportForInterface,TorqueManoeuvrability,TrainedDeployableAmount,Transporter,TurretNoneID,TurretNoneType,TurretOneID,TurretOneType,TurretThreeID,TurretThreeType,TurretTwoID,TurretTwoType,TypeForAcknow,UnitDescriptorID,UnitMovingType,UnitStealthBonus,UpgradeRequire,UpwardsSpeed,VehicleSubType,VeteranDeployableAmount,VisibilityModuleID,VisibilitySelectorID,VitesseCombat,WeaponFourAffecteParNombre,WeaponFourAffichageMenu,WeaponFourAffichageMunitionParSalve,WeaponFourAmmunitionID,WeaponFourAngleDispersion,WeaponFourArme,WeaponFourCaliber,WeaponFourCorrectedShotDispersionMultiplier,WeaponFourDescriptorId,WeaponFourDispersionAtMaxRange,WeaponFourDispersionAtMinRange,WeaponFourEfficaciteSelonPortee,WeaponFourFX_frottement,WeaponFourFX_tir_sans_physic,WeaponFourFX_tir_tendu,WeaponFourFX_vitesse_de_depart,WeaponFourFireDescriptor,WeaponFourFireTriggeringProbability,WeaponFourGuidance,WeaponFourHitRollRule,WeaponFourIgnoreInflammabilityConditions,WeaponFourInterdireTirReflexe,WeaponFourInterfaceWeaponTexture,WeaponFourIsFireAndForget,WeaponFourIsSubAmmunition,WeaponFourLevel,WeaponFourMissileDescriptor,WeaponFourMissileTimeBetweenCorrections,WeaponFourName,WeaponFourNbTirParSalves,WeaponFourNbrProjectilesSimultanes,WeaponFourNeedModelChange,WeaponFourNoiseDissimulationMalus,WeaponFourPhysicalDamages,WeaponFourPorteeMaximale,WeaponFourPorteeMaximaleBateaux,WeaponFourPorteeMaximaleHA,WeaponFourPorteeMaximaleProjectile,WeaponFourPorteeMaximaleTBA,WeaponFourPorteeMinimale,WeaponFourPorteeMinimaleBateaux,WeaponFourPorteeMinimaleHA,WeaponFourPorteeMinimaleProjectile,WeaponFourPorteeMinimaleTBA,WeaponFourProjectileType,WeaponFourPuissance,WeaponFourRadiusSplashPhysicalDamages,WeaponFourRadiusSplashSuppressDamages,WeaponFourRandomDispersion,WeaponFourRayonPinned,WeaponFourSmokeDescriptor,WeaponFourSupplyCost,WeaponFourSuppressDamages,WeaponFourTempsAnimation,WeaponFourTempsDeVisee,WeaponFourTempsEntreDeuxFx,WeaponFourTempsEntreDeuxSalves,WeaponFourTempsEntreDeuxTirs,WeaponFourTirIndirect,WeaponFourTirReflexe,WeaponFourTypeArme,WeaponFourTypeName,WeaponFourWeaponCursorType,WeaponManagerID,WeaponManagerModuleSelectorID,WeaponOneAffecteParNombre,WeaponOneAffichageMenu,WeaponOneAffichageMunitionParSalve,WeaponOneAmmunitionID,WeaponOneAngleDispersion,WeaponOneAnimateOnlyOneSoldier,WeaponOneArme,WeaponOneCaliber,WeaponOneCorrectedShotDispersionMultiplier,WeaponOneDescriptorId,WeaponOneDispersionAtMaxRange,WeaponOneDispersionAtMinRange,WeaponOneEfficaciteSelonPortee,WeaponOneFX_frottement,WeaponOneFX_tir_sans_physic,WeaponOneFX_tir_tendu,WeaponOneFX_vitesse_de_depart,WeaponOneFireDescriptor,WeaponOneFireTriggeringProbability,WeaponOneGuidance,WeaponOneHitRollRule,WeaponOneIgnoreInflammabilityConditions,WeaponOneInterdireTirReflexe,WeaponOneInterfaceWeaponTexture,WeaponOneIsFireAndForget,WeaponOneIsSubAmmunition,WeaponOneLevel,WeaponOneMissileDescriptor,WeaponOneMissileTimeBetweenCorrec

In [41]:
leading_cols = ['UnitName', 'MotherCountry', 'ProductionPrice', 'ProductionYear',
                'MaxDamages', 'IsPrototype',
                'ManageUnitOrientation', 'CanDeploySmoke', 
                'CanWinExperience', 'ExperienceGainBySecond',
                'UnitMovingType', 'MovementType', 'SpeedBonusOnRoad', 'Maxspeed', 'FuelCapacity', 'FuelMoveDuration',
                'Transporter', 'IsTargetableAsBoat', 'TerrainsToIgnoreMask',
                'RookieDeployableAmount', 'TrainedDeployableAmount', 'HardenedDeployableAmount',
                'VeteranDeployableAmount', 'EliteDeployableAmount',
                'UnitStealthBonus', 'HitRollSizeModifier', 'HitRollECMModifier',
                'Maxspeed', 'UnitMovingType', 'FlyingAltitude', 'MinimalAltitude', 'GunMuzzleSpeed',
                'CyclicManoeuvrability', 'GFactorLimit', 'LateralSpeed', 'Mass', 'MaxInclination',
                'RotorArea', 'TorqueManoeuvrability', 'UpwardsSpeed', 'TempsDemiTour',
                'MaxAcceleration', 'MaxDeceleration', 'WeaponSabordAngle',
                'AliasName', 'ClassNameForDebug', '_ShortDatabaseName', 'NameInMenuToken',
                'Factory', 'IconeType', 'MaxPacks', 'Nationalite', 'PositionInMenu', 'TypeForAcknow',
                'VehicleSubType', 'Key', 'CoutEtoile', 'UpgradeRequire',                
                'ProductionTime',
                'Category', 'AcknowUnitType', 'TypeForAcknow', 'DescriptorId',
                'MaxHPForHUD',                
               ]
nonleading_cols = list(set(wargame.columns) - set(leading_cols))
wargame = wargame.reindex(columns=leading_cols + nonleading_cols)

In [46]:
wargame['IsPrototype'] = wargame['IsPrototype'].map(lambda v: True if pd.notnull(v) else False)

In [49]:
wargame['MotherCountry'].value_counts()

URSS    205
US      173
POL     108
RDA     106
CHI     101
NK      100
TCH      97
ISR      96
UK       95
FR       92
RFA      88
ROK      80
HOL      80
ANZ      72
CAN      71
SWE      67
JAP      66
DAN      61
NOR      60
Name: MotherCountry, dtype: int64

In [53]:
wargame[wargame['MotherCountry'] == 'RFA'].head()

,UnitName,MotherCountry,ProductionPrice,ProductionYear,MaxDamages,IsPrototype,ManageUnitOrientation,CanDeploySmoke,CanWinExperience,ExperienceGainBySecond,UnitMovingType,MovementType,SpeedBonusOnRoad,Maxspeed,FuelCapacity,FuelMoveDuration,Transporter,IsTargetableAsBoat,TerrainsToIgnoreMask,RookieDeployableAmount,TrainedDeployableAmount,HardenedDeployableAmount,VeteranDeployableAmount,EliteDeployableAmount,UnitStealthBonus,HitRollSizeModifier,HitRollECMModifier,Maxspeed,UnitMovingType,FlyingAltitude,MinimalAltitude,GunMuzzleSpeed,CyclicManoeuvrability,GFactorLimit,LateralSpeed,Mass,MaxInclination,RotorArea,TorqueManoeuvrability,UpwardsSpeed,TempsDemiTour,MaxAcceleration,MaxDeceleration,WeaponSabordAngle,AliasName,ClassNameForDebug,_ShortDatabaseName,NameInMenuToken,Factory,IconeType,MaxPacks,Nationalite,PositionInMenu,TypeForAcknow,VehicleSubType,Key,CoutEtoile,UpgradeRequire,ProductionTime,Category,AcknowUnitType,TypeForAcknow,DescriptorId,MaxHPForHUD,WeaponThreeFX_frottement,WeaponTwoPorteeMinimale,WeaponFourPorteeMinimaleBateaux,WeaponThreeInterdireTirReflexe,WeaponTwoSalvoStockIndex_ForInterface,WeaponThreeTempsAnimation,WeaponTwoIsFireAndForget,WeaponTwoSuppressDamages,WeaponOneDispersionAtMinRange,WeaponTwoTempsDeVisee,WeaponOneSalvoStockIndex,WeaponTwoTypeName,WeaponManagerModuleSelectorID,WeaponTwoTirReflexe,WeaponOneNoiseDissimulationMalus,WeaponThreeSalvoStockIndex_ForInterface,WeaponFourFX_tir_sans_physic,WeaponOneRayonPinned,WeaponFourPorteeMinimaleHA,WeaponTwoPorteeMinimaleTBA,WeaponOnePorteeMaximaleTBA,WeaponOnePorteeMaximale,WeaponFourInterdireTirReflexe,KillExperienceBonus,WeaponOneFX_tir_sans_physic,TextureForInterface,WeaponFourTempsEntreDeuxSalves,WeaponFourCaliber,WeaponThreePorteeMinimaleProjectile,WeaponOneIgnoreInflammabilityConditions,WeaponTwoSmokeDescriptor,WeaponThreeMissileTimeBetweenCorrections,WeaponOneAmmunitionID,WeaponOneTempsDeVisee,WeaponOneGuidance,WeaponTwoFX_frottement,WeaponThreeDescriptorId,WeaponOneInterdireTirReflexe,WeaponThreeIsFireAndForget,VisibilitySelectorID,WeaponOneFireDescriptor,WeaponOneCaliber,WeaponFourTypeArme,WeaponTwoPorteeMaximaleTBA,WeaponTwoRayonPinned,WeaponTwoCaliber,WeaponOneRadiusSplashSuppressDamages,WeaponTwoFX_tir_tendu,WeaponTwoTypeArme,WeaponOneMissileDescriptor,WeaponOneTirContinu,WeaponThreePorteeMinimale,WeaponOneProjectileType,WeaponFourPorteeMaximaleProjectile,WeaponThreeEfficaciteSelonPortee,TurretTwoID,WeaponFourAngleDispersion,WeaponTwoAffecteParNombre,WeaponThreePorteeMaximaleBateaux,WeaponThreeName,WeaponTwoNeedModelChange,WeaponOneFireTriggeringProbability,WeaponOneName,WeaponOnePorteeMinimaleHA,WeaponFourFireTriggeringProbability,WeaponOneWeaponCursorType,WeaponFourIsFireAndForget,WeaponThreeTypeName,WeaponThreeTempsEntreDeuxFx,WeaponThreeTempsEntreDeuxTirs,WeaponThreeTempsEntreDeuxSalves,WeaponOnePuissance,WeaponThreeProjectileType,TurretNoneType,WeaponThreeFX_vitesse_de_depart,SalvoWeaponOneIsMain,WeaponFourInterfaceWeaponTexture,WeaponThreeMissileDescriptor,WeaponFourTempsAnimation,WeaponThreePuissance,WeaponFourPorteeMinimaleTBA,WeaponThreeRayonPinned,WeaponThreeFireDescriptor,WeaponThreeTirContinu,WeaponOneFX_tir_tendu,WeaponFourWeaponCursorType,WeaponOneIsSubAmmunition,WeaponOneAnimateOnlyOneSoldier,WeaponThreePorteeMinimaleBateaux,TurretNoneID,WeaponTwoProjectileType,WeaponFourProjectileType,WeaponTwoWeaponCursorType,WeaponFourSuppressDamages,WeaponOnePorteeMaximaleProjectile,WeaponTwoDispersionAtMinRange,WeaponOneTirReflexe,WeaponFourFX_vitesse_de_depart,WeaponThreeLevel,WeaponManagerID,WeaponTwoDescriptorId,TurretThreeID,MovementSelectorID,WeaponThreeDispersionAtMinRange,WeaponFourPorteeMaximale,WeaponThreeAnimateOnlyOneSoldier,WeaponOnePorteeMinimaleProjectile,WeaponThreeTirReflexe,WeaponOneTempsEntreDeuxTirs,WeaponFourArme,WeaponFourMissileTimeBetweenCorrections,WeaponFourRandomDispersion,WeaponThreeAffichageMenu,WeaponOneTempsEntreDeuxFx,WeaponFourNeedModelChange,WeaponFourRadiusSplashPhysicalDamages,WeaponThreeFireTriggeringProbability

In [61]:
mc_map = {
    'URSS': 'Soviet Union',
    'US': 'United States',
    'RDA': 'East Germany',
    'POL': 'Poland',
    'CHI': 'China',
    'NK': 'North Korea',
    'TCH': 'Czechoslovakia',
    'ISR': 'Israel',
    'UK': 'United Kingdom',
    'FR': 'France',
    'RFA': 'West Germany',
    'ROK': 'South Korea',
    'HOL': 'Netherlands',
    'ANZ': 'ANZAC',
    'CAN': 'Canada',
    'SWE': 'Sweden',
    'JAP': 'Japan',
    'DAN': 'Denmark',
    'NOR': 'Norway'    
}

wargame['MotherCountry'] = wargame['MotherCountry'].map(mc_map)

In [63]:
wargame['MovementType'].value_counts(dropna=False)

TMouvementHandlerLandVehicleDescriptor    1370
TMouvementHandlerAirplaneDescriptor        229
TMouvementHandlerHelicopterDescriptor      219
Name: MovementType, dtype: int64

In [65]:
mv_map = {
    'TMouvementHandlerLandVehicleDescriptor': 'Land',
    'TMouvementHandlerAirplaneDescriptor': 'Airplane',
    'TMouvementHandlerHelicopterDescriptor': 'Helicopter'
}

wargame['MovementType'] = wargame['MovementType'].map(mv_map)

In [69]:
wargame['Transporter'] = wargame['Transporter'].map(lambda v: False if pd.isnull(v) else True)
wargame['IsTargetableAsBoat'] = wargame['IsTargetableAsBoat'].map(lambda v: False if pd.isnull(v) else True)

In [72]:
wargame = wargame.rename(columns={'MaxDamages': 'Health'})

In [74]:
wargame = wargame.rename(columns={'IsPrototype': 'Prototype'})

## Undecided question:

Which of these stats is the real one?

In [78]:
wargame['ManageUnitOrientation'].value_counts(dropna=False)

True    1522
NaN      296
Name: ManageUnitOrientation, dtype: int64

In [79]:
wargame['AutoOrientation'].value_counts(dropna=False)

True    1786
NaN       32
Name: AutoOrientation, dtype: int64

In [88]:
wargame = wargame.T.drop_duplicates().T

In [93]:
wargame['HitRollSizeModifier'] = wargame['HitRollSizeModifier'].map(lambda v: v if pd.notnull(v) else 0).astype(float)
wargame['HitRollECMModifier'] = wargame['HitRollECMModifier'].map(lambda v: v if pd.notnull(v) else 0).astype(float)

## Undecided question:

What does this mean?

In [99]:
wargame['TerrainsToIgnoreMask'].value_counts()

26    10
16    10
24    10
Name: TerrainsToIgnoreMask, dtype: int64

In [100]:
del wargame['UpgradeRequire']

In [103]:
del wargame['TextureForInterface']

In [94]:
wargame.head(5)

,UnitName,MotherCountry,ProductionPrice,ProductionYear,Health,Prototype,ManageUnitOrientation,CanDeploySmoke,CanWinExperience,ExperienceGainBySecond,UnitMovingType,MovementType,SpeedBonusOnRoad,Maxspeed,FuelCapacity,FuelMoveDuration,Transporter,IsTargetableAsBoat,TerrainsToIgnoreMask,RookieDeployableAmount,TrainedDeployableAmount,HardenedDeployableAmount,VeteranDeployableAmount,EliteDeployableAmount,UnitStealthBonus,HitRollSizeModifier,HitRollECMModifier,FlyingAltitude,MinimalAltitude,GunMuzzleSpeed,CyclicManoeuvrability,GFactorLimit,LateralSpeed,Mass,MaxInclination,RotorArea,TorqueManoeuvrability,UpwardsSpeed,TempsDemiTour,MaxAcceleration,MaxDeceleration,WeaponSabordAngle,AliasName,ClassNameForDebug,_ShortDatabaseName,NameInMenuToken,Factory,IconeType,MaxPacks,Nationalite,PositionInMenu,TypeForAcknow,VehicleSubType,Key,CoutEtoile,UpgradeRequire,ProductionTime,Category,AcknowUnitType,DescriptorId,MaxHPForHUD,WeaponThreeFX_frottement,WeaponTwoPorteeMinimale,WeaponTwoSalvoStockIndex_ForInterface,WeaponTwoIsFireAndForget,WeaponTwoSuppressDamages,WeaponOneDispersionAtMinRange,WeaponTwoTempsDeVisee,WeaponOneSalvoStockIndex,WeaponTwoTypeName,WeaponManagerModuleSelectorID,WeaponTwoTirReflexe,WeaponOneNoiseDissimulationMalus,WeaponOneRayonPinned,WeaponOnePorteeMaximaleTBA,WeaponOnePorteeMaximale,KillExperienceBonus,WeaponOneFX_tir_sans_physic,TextureForInterface,WeaponFourTempsEntreDeuxSalves,WeaponFourCaliber,WeaponOneIgnoreInflammabilityConditions,WeaponThreeMissileTimeBetweenCorrections,WeaponOneAmmunitionID,WeaponOneTempsDeVisee,WeaponOneGuidance,WeaponTwoFX_frottement,WeaponThreeDescriptorId,VisibilitySelectorID,WeaponOneFireDescriptor,WeaponOneCaliber,WeaponFourTypeArme,WeaponTwoPorteeMaximaleTBA,WeaponTwoRayonPinned,WeaponTwoCaliber,WeaponOneRadiusSplashSuppressDamages,WeaponTwoFX_tir_tendu,WeaponTwoTypeArme,WeaponOneMissileDescriptor,WeaponOneTirContinu,WeaponThreePorteeMinimale,WeaponOneProjectileType,WeaponThreeEfficaciteSelonPortee,TurretTwoID,WeaponFourAngleDispersion,WeaponTwoAffecteParNombre,WeaponThreePorteeMaximaleBateaux,WeaponThreeName,WeaponTwoNeedModelChange,WeaponOneFireTriggeringProbability,WeaponOneName,WeaponOnePorteeMinimaleHA,WeaponFourFireTriggeringProbability,WeaponOneWeaponCursorType,WeaponThreeTypeName,WeaponThreeTempsEntreDeuxFx,WeaponThreeTempsEntreDeuxTirs,WeaponThreeTempsEntreDeuxSalves,WeaponOnePuissance,WeaponThreeProjectileType,TurretNoneType,WeaponThreeFX_vitesse_de_depart,SalvoWeaponOneIsMain,WeaponFourInterfaceWeaponTexture,WeaponThreeMissileDescriptor,WeaponThreePuissance,WeaponThreeRayonPinned,WeaponThreeFireDescriptor,WeaponOneFX_tir_tendu,WeaponFourWeaponCursorType,WeaponOneIsSubAmmunition,WeaponOneAnimateOnlyOneSoldier,TurretNoneID,WeaponTwoProjectileType,WeaponTwoWeaponCursorType,WeaponFourSuppressDamages,WeaponOnePorteeMaximaleProjectile,WeaponTwoDispersionAtMinRange,WeaponOneTirReflexe,WeaponFourFX_vitesse_de_depart,WeaponThreeLevel,WeaponManagerID,WeaponTwoDescriptorId,TurretThreeID,MovementSelectorID,WeaponFourPorteeMaximale,WeaponThreeTirReflexe,WeaponOneTempsEntreDeuxTirs,WeaponFourArme,WeaponOneTempsEntreDeuxFx,WeaponFourRadiusSplashPhysicalDamages,WeaponThreeFireTriggeringProbability,WeaponFourLevel,TurretTwoType,WeaponFourPuissance,WeaponThreeCaliber,WeaponOneAffecteParNombre,WeaponTwoRadiusSplashSuppressDamages,WeaponThreeNbTirParSalves,WeaponTwoTempsAnimation,WeaponOneRadiusSplashPhysicalDamages,WeaponOneIsFireAndForget,WeaponTwoRadiusSplashPhysicalDamages,WeaponOneAngleDispersion,WeaponThreeRadiusSplashSuppressDamages,WeaponTwoNoiseDissimulationMalus,TurretOneID,WeaponThreeRadiusSplashPhysicalDamages,WeaponFourTypeName,WeaponFourTempsDeVisee,DamageModuleID,WeaponOneNbTirParSalves,WeaponTwoTempsEntreDeuxFx,MovementModuleID,WeaponThreeSuppressDamages,WeaponTwoArme,WeaponTwoAngleDispersion,WeaponFourSupplyCost,WeaponThreeFX_tir_tendu,WeaponOneEfficaciteSelonPortee,TextureMotherCountryForInterface,WeaponOneMissileTimeBetweenCorrections,WeaponOneAffichageMunitionParSalve,WeaponThree

In [ ]:
wargame['AcknowUnitType'].value_counts().plot(kind='bar')